In [13]:
# First instal:
#conda install -c huggingface transformers
#conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch
#conda create -n tf tensorflow
#conda activate tf

In [14]:
#! pip freeze

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [2]:
tokenizer = AutoTokenizer.from_pretrained("tartuNLP/EstBERT")
model = AutoModelForMaskedLM.from_pretrained("tartuNLP/EstBERT")

In [11]:
bert_tagger = BertTagger(bert_location='tartuNLP/EstBERT')

Some weights of BertModel were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import torch
from typing import MutableMapping, List
from transformers import BertTokenizer, logging, BertModel

logging.set_verbosity(30)
from estnltk.text import Text
from estnltk.taggers import Tagger
from estnltk.layer.layer import Layer
import numpy as np


class BertTagger(Tagger):
    """Tags BERT embeddings."""

    def __init__(self, bert_location: str, sentences_layer: str = 'sentences',
                 token_level: bool = False,
                 output_layer: str = 'bert_embeddings', bert_layers: List[int] = None, method='concatenate'):

        if bert_layers is None:
            bert_layers = [-4, -3, -2, -1]
        else:
            for layer in bert_layers:
                if abs(layer) > 12:
                    msg = "BERT base model only has 12 layers of transformer encoder, chose layers from (-12..-1). It " \
                          "is reasonable to choose layers from the last layers, for example [-4, -3, -2, -1]: last 4 " \
                          "layers. "
                    raise Exception(msg)
        self.conf_param = ('bert_location', 'bert_model', 'tokenizer', 'method', 'token_level', 'bert_layers')
        if bert_location is None:
            msg = "Directory containing BERT model must be specified."
            raise Exception(msg)
        else:
            self.bert_location = bert_location
        if method not in ('concatenate', 'add', 'all'):
            msg = "Method can be 'concatenate', 'add' or 'all'."
            raise Exception(msg)
        self.method = method
        self.output_layer = output_layer
        self.input_layers = [sentences_layer]

        self.bert_model = BertModel.from_pretrained(bert_location, output_hidden_states=True)

        self.tokenizer = BertTokenizer.from_pretrained(self.bert_location)

        self.output_attributes = ['token', 'bert_embedding']

        self.token_level = token_level
        self.bert_layers = bert_layers

    def _make_layer(self, text: Text, layers: MutableMapping[str, Layer], status: dict) -> Layer:
        sentences_layer = layers[self.input_layers[0]]
        embeddings_layer = Layer(name=self.output_layer, text_object=text, attributes=self.output_attributes,
                                 ambiguous=True)

        start, i = 0, 0
        word_spans = []

        for k, sentence in enumerate(sentences_layer):

            for word in sentence:
                word_spans.append((word.start, word.end, word.text))
            sent_text = sentence.enclosing_text

            embeddings = get_embeddings(sent_text, self.bert_model, self.tokenizer, self.method, self.bert_layers)[
                         1:-1]  # first one in start token, and last one is sep token
            tokens = self.tokenizer.tokenize(sent_text)
            assert len(tokens) == len(embeddings)
            if k != 0:  # move the start manually when next sentence starts
                start = word_spans[i][0]

            if self.token_level:  # annotates tokens
                for j, packed in enumerate(zip(embeddings, tokens)):
                    token_emb, token_init = packed[0], packed[1]

                    if not token_init.startswith("#") and j != 0:  # move to next word
                        if start == word_spans[i][1]:  # BERT's wordpiece tokenizer can tokenize differently
                            i += 1  # next word starts
                            word_span = word_spans[i]
                            start = word_span[0]  # the start id of this word
                    if self.method == 'all':
                        embedding = []
                        for tok_emb in token_emb:
                            emb = []
                            for e in tok_emb:
                                emb.append(float(e))
                            embedding.append(emb)
                    else:
                        embedding = [float(t) for t in token_emb]

                    attributes = {'token': token_init, 'bert_embedding': embedding}
                    token = token_init.strip()
                    embeddings_layer.add_annotation((start, start + len(token.replace('#', ''))), **attributes)
                    start += len(token.replace('#', ''))  # adding token length to the current pointer

                i += 1  # move the pointer manually

            else:  # annotates full words, adding the token level embedding together
                collected_tokens = []
                collected_embeddings = []
                for j, packed in enumerate(zip(embeddings, tokens)):
                    token_emb, token_init = packed[0], packed[1]

                    if i == 0 and j == 0:
                        start = 0
                        collected_embeddings.append(token_emb)
                        collected_tokens.append(token_init)
                        start += len(token_init)

                    if i != 0 and j == 0:
                        collected_tokens = [token_init]
                        collected_embeddings = [token_emb]
                        start = word_spans[i][0]

                        start += len(token_init)

                    if not token_init.startswith("#") and j != 0:  # move to next word
                        if start == word_spans[i][1]:  # BERT's wordpiece tokenizer can tokenize differently

                            if collected_embeddings:
                                if self.method == 'all':
                                    embedding = []
                                    for tok_embs in collected_embeddings:
                                        token_embs = []
                                        for embs in tok_embs:
                                            token_embs_emb = []
                                            for emb in embs:
                                                token_embs_emb.append(float(emb))
                                            token_embs.append(token_embs_emb)
                                        embedding.append(token_embs)
                                else:
                                    embedding = [float(t) for t in np.sum(collected_embeddings, 0)]

                                attributes = {'token': collected_tokens, 'bert_embedding': embedding}
                                embeddings_layer.add_annotation((word_spans[i][0], word_spans[i][1]),
                                                                **attributes)

                            i += 1  # next word starts
                            start = word_spans[i][0]  # the start id of this word
                            collected_embeddings = [token_emb]
                            collected_tokens = [token_init]
                            start += len(token_init)
                        else:
                            start += len(token_init)
                            collected_tokens.append(token_init)
                            collected_embeddings.append(token_emb)
                    elif token_init.startswith("#"):
                        collected_tokens.append(token_init)
                        collected_embeddings.append(token_emb)
                        start += len(token_init.replace("#", ''))

                if collected_tokens:
                    if self.method == 'all':
                        embedding = []
                        for tok_embs in collected_embeddings:
                            token_embs = []
                            for embs in tok_embs:
                                token_embs_emb = []
                                for emb in embs:
                                    token_embs_emb.append(float(emb))
                                token_embs.append(token_embs_emb)
                            embedding.append(token_embs)
                    else:
                        embedding = [float(t) for t in np.sum(collected_embeddings, 0)]

                    attributes = {'token': collected_tokens, 'bert_embedding': embedding}
                    embeddings_layer.add_annotation((word_spans[i][0], word_spans[i][1]),
                                                    **attributes)

                i += 1  # move the pointer manually

        return embeddings_layer


def get_embeddings(sentence: str, model, tokenizer, method, bert_layers):
    input_data = tokenizer.encode_plus(sentence)
    input_ids = input_data.get('input_ids')
    token_vecs_cat = []
    if len(input_ids) > 512:  # maximum sequence length can be 512
        msg = "Input sentence is too big (%s), splitting the sentence." % len(input_ids)
        print(msg)
        collected_input_ids = []
        while True:
            collected_input_ids.append(input_ids[:512])
            input_ids = input_ids[512:]
            if len(input_ids) <= 512:
                collected_input_ids.append(input_ids)
                break
    else:
        collected_input_ids = [input_ids]

    for i, input_ids in enumerate(collected_input_ids):

        segments_ids = [1] * len(input_ids)
        tokens_tensor = torch.tensor([input_ids])
        segments_tensors = torch.tensor([segments_ids])

        with torch.no_grad():
            outputs = model(tokens_tensor, segments_tensors)
            hidden_states = outputs[2]
        token_embeddings = torch.stack(hidden_states, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1, 0, 2)

        for token in token_embeddings:
            if method == 'concatenate':  # concatenate the vectors
                layers = [token[i] for i in bert_layers]
                cat_vec = torch.cat(layers, dim=0)
                token_vecs_cat.append(np.asarray(cat_vec))

            if method == 'add':  # elementwise addition
                layers = [np.asarray(token[i]) for i in bert_layers]
                sum_vec = np.sum(layers, 0)
                token_vecs_cat.append(np.asarray(sum_vec))

            if method == 'all':  # return all
                layers = [np.asarray(token[i]) for i in bert_layers]
                token_vecs_cat.append(layers)

    return token_vecs_cat

In [5]:
import pandas as pd
data = pd.read_csv("Pohitabel.csv")
data.head()

,Kohtuasja_ID,Kohtuasja_nr,Lõigu sisu,Pealkiri1,Tag
0,0,C‑416/20 PPU,EUROOPA KOHTU OTSUS (neljas koda),Koda,C19Centre
1,0,C‑416/20 PPU,17. detsember 2020,Kuupaev,C19Centre
2,0,C‑416/20 PPU,Eelotsusetaotlus – Eelotsuse kiirmenetlus – Po...,Viited,C71Indicateur
3,0,C‑416/20 PPU,TR,Poolte_nimed,C02AlineaAltA
4,0,C‑416/20 PPU,"Generalstaatsanwaltschaft Hamburg,",Poolte_nimed,C02AlineaAltA


In [6]:
column_list=data['Lõigu sisu'].to_list()
column_list[0:100]

['EUROOPA KOHTU OTSUS (neljas koda)',
 '17. detsember 2020',
 'Eelotsusetaotlus – Eelotsuse kiirmenetlus – Politseikoostöö ja õigusalane koostöö kriminaalasjades – Raamotsus 2002/584/JSK – Euroopa vahistamismäärus – Artikli 4a lõige 1 – Liikmesriikidevaheline üleandmiskord – Täitmise tingimused – Täitmata jätmise vabatahtlikud alused – Erandid – Täitmise kohustuslikkus – Tagaselja mõistetud karistus – Kahtlustatava või süüdistatava põgenemine – Direktiiv (EL) 2016/343 – Artiklid 8 ja 9 – Õigus viibida kohtulikul arutelul – Nõuded tagaselja süüdimõistmise puhul – Kontrollimine süüdimõistetu üleandmisel',
 'TR',
 'Generalstaatsanwaltschaft Hamburg,',
 'saksa',
 '–        Generalstaatsanwaltschaft Hamburg, esindaja: J. Fröhlich,',
 '–        Saksamaa valitsus, esindajad: J. Möller, M. Hellmann ja F. Halabi,',
 '–        Rumeenia valitsus, esindajad: E. Gane, L.‑E. Batagoi ja A. Wellman,',
 '–        Poola valitsus, esindajad: B. Majczyna ja J. Sawicka,',
 '–        Euroopa Komisjon, esind

In [7]:
column_list_low=[str(x).lower() for x in column_list]   
column_list_low[0:100]

['euroopa kohtu otsus (neljas koda)',
 '17. detsember 2020',
 'eelotsusetaotlus – eelotsuse kiirmenetlus – politseikoostöö ja õigusalane koostöö kriminaalasjades – raamotsus 2002/584/jsk – euroopa vahistamismäärus – artikli 4a lõige 1 – liikmesriikidevaheline üleandmiskord – täitmise tingimused – täitmata jätmise vabatahtlikud alused – erandid – täitmise kohustuslikkus – tagaselja mõistetud karistus – kahtlustatava või süüdistatava põgenemine – direktiiv (el) 2016/343 – artiklid 8 ja 9 – õigus viibida kohtulikul arutelul – nõuded tagaselja süüdimõistmise puhul – kontrollimine süüdimõistetu üleandmisel',
 'tr',
 'generalstaatsanwaltschaft hamburg,',
 'saksa',
 '–        generalstaatsanwaltschaft hamburg, esindaja: j. fröhlich,',
 '–        saksamaa valitsus, esindajad: j. möller, m. hellmann ja f. halabi,',
 '–        rumeenia valitsus, esindajad: e. gane, l.‑e. batagoi ja a. wellman,',
 '–        poola valitsus, esindajad: b. majczyna ja j. sawicka,',
 '–        euroopa komisjon, esind

In [41]:
#text = Text(column_list_low[0])
#text.analyse('segmentation')

Text(text='euroopa kohtu otsus (neljas koda)')

In [ ]:
#text = Text("Aga mulle tundub, et kogu maailm ootab muusikamaailmalt midagi erutavalt uut minimalismi kõrvale. Tere tulemast")
#text.analyse('segmentation')

In [42]:
#bert_tagger.tag(text)

Text(text='euroopa kohtu otsus (neljas koda)')

In [63]:
#frame=text.bert_embeddings
#frame

Layer(name='bert_embeddings', attributes=('token', 'bert_embedding'), spans=SL[Span('euroopa', [{'token': ['euroopa'], 'bert_embedding': [-0.5070216655731201, 0.8251616358757019, 0.08608068525791168, -0.7443538904190063, 0.42327311635017395, -0.3357990086078644, -0.1661747545003891, 0.17598856985569, 0.6337613463401794, 0.24088409543037415, 0.6563606858253479, -0.03960776329040527, -0.45246443152427673, 0.8547276258468628, 0.472175657749176, -0.28201010823249817, -0.7163088917732239, 0.006684646010398865, 0.24420717358589172, -1.4477111101150513, 0.48714497685432434, -0.6535736918449402, -0.5764666795730591, 0.4968271851539612, 0.5367496609687805, -0.31698963046073914, -0.48657453060150146, -0.08248171210289001, 0.4301958680152893, -1.090407133102417, -0.08540712296962738, 0.18112631142139435, -0.24123120307922363, -0.004547242075204849, 0.08319372683763504, -0.08306585252285004, -0.1008380651473999, 0.4537557363510132, -0.2833148241043091, 0.350974440574646, 0.8564369082450867, 0.45465800166130066, -0.7051980495452881, 0.5887426733970642, -0.14422693848609924, -0.2075478434562683, -0.1192743331193924, -0.26753631234169006, -0.2809869050979614, 0.6301348805427551, -0.03228113427758217, 0.11884523928165436, -0.4739339053630829, 1.6283085346221924, -0.8073915839195251, 0.06215781345963478, 0.24660812318325043, 0.6828106045722961, 0.21392515301704407, 0.33367466926574707, 0.7278926372528076, 0.1813514530658722, 0.3288612961769104, -0.28021177649497986, -0.6887204647064209, -0.47454333305358887, -0.8661482930183411, 0.4904363751411438, -0.5963777899742126, 1.2196552753448486, -0.005253531038761139, 0.9548414349555969, -1.288742184638977, 0.34009841084480286, 0.022736679762601852, 0.8805800080299377, 0.43039748072624207, 0.33595964312553406, 0.9639607667922974, 0.15424798429012299, 0.10911581665277481, -0.9129564166069031, 0.3496178388595581, 0.9849584698677063, -0.03565488010644913, 0.2728867828845978, -0.11836814135313034, -0.22831490635871887, 0.37243324518203735, 0.6129550337791443, -0.29399263858795166, 0.12684011459350586, 0.9221431016921997, -1.3585461378097534, 0.532674252986908, 0.30425938963890076, 0.3346153497695923, 0.6102094054222107, 0.895257294178009, -0.5187780857086182, -2.270247459411621, 0.48431989550590515, 0.16549566388130188, -1.009299874305725, -0.052820414304733276, 0.6967551112174988, -0.222488135099411, -0.3365393280982971, -0.6888496279716492, -0.08834625780582428, 0.8404138684272766, 0.27092260122299194, -0.06350992619991302, 0.17027419805526733, -0.5221251845359802, -0.09260865300893784, 0.02734369784593582, -0.7704155445098877, -0.6259997487068176, 0.9132505655288696, -0.16971565783023834, 0.30419275164604187, -0.8628953695297241, -0.05724005401134491, 0.5123506784439087, -0.016252726316452026, -0.29188069701194763, -0.29007166624069214, 0.34348592162132263, -0.4541884660720825, 0.08117569983005524, 0.8810927271842957, 0.5833972096443176, 0.506653368473053, -0.2998389005661011, 0.5207535624504089, -0.8357948660850525, 0.2776525318622589, 0.13548947870731354, -0.4728220999240875, -0.13969480991363525, 0.19873909652233124, 0.31471699476242065, -0.20381280779838562, 0.0757228285074234, -0.5954983830451965, -0.49431657791137695, -0.2358124852180481, -0.2696155309677124, 0.9700067639350891, -0.5716537237167358, 0.2764686942100525, 0.19426043331623077, -0.19349369406700134, -0.25277888774871826, -1.5308090448379517, -0.050041839480400085, 0.2041982263326645, -0.15223756432533264, -0.6272586584091187, 0.3379376232624054, -0.9533262252807617, -1.370027780532837, 0.2159697711467743, -0.890117883682251, 0.13204419612884521, -0.6092806458473206, 0.6566190123558044, 0.6296209096908569, 0.18166857957839966, 0.820346474647522, -0.11210018396377563, -0.2113056182861328, 0.034647136926651, -0.004108726978302002, -0.013059910386800766, 0.2846853733062744, -0.7096424698829651, 0.068302683532238, 0.48376771807670593, -0.1539108157157898, -0.8307573199272156, 0.1825527548789978, -0.9772115349769592, -0.14318767189979553, 

In [71]:
#len(frame['text'])


7

In [72]:
#np.sum(frame['bert_embedding'], axis=0) # saada ühe lõigu summeeritud vektor

array([[-3.56959184,  3.38594219, -0.3977652 , ...,  0.73575374,
         4.82716704,  1.92730936]])

In [84]:
#len(column_list_low)

1057199

In [23]:
# koondab iga sõna vektorid üheks ning lisab need ühte listi
row_nr=[]
vektor=[]
i=0
while i < (len(column_list_low)-1050000):
    text = Text(column_list_low[i])
    text.analyse('segmentation')
    bert_tagger.tag(text)
    frame = text.bert_embeddings
    listike = np.sum(frame['bert_embedding'], axis=0)
    row_nr.append(i)
    vektor.append(listike[0])
    i += 1
print(row_nr)
print(vektor)

C:\Users\sirle\anaconda3\envs\py37\lib\site-packages\numpy\core\fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


TypeError: unsupported operand type(s) for +: 'ImmutableList' and 'ImmutableList'

In [24]:
#testandmed
test_text = Text("Käive käib käibe juurde")
test_text.analyse('segmentation')
bert_tagger.tag(test_text)
frame = test_text.bert_embeddings
proovivektoriks1=np.sum(frame["bert_embedding"], axis=0)

In [25]:
def vordlus (vektor, row_nr, proovivektor, column_list_low):
    sarnased_maatriks=cosine_similarity(np.stack(vektor, axis=0), Y=proovivektor.reshape(1, -1), dense_output=True)
    sorteeritud_indeksid=np.argsort(np.squeeze(sarnased_maatriks))
    koige_sarnasem_idx=row_nr[sorteeritud_indeksid[len(sorteeritud_indeksid) - 1]]
    return column_list_low[koige_sarnasem_idx]

vordlus(vektor, row_nr, proovivektoriks1, column_list_low)

'jcm europe (uk) ltd'